In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
#set excecutable path

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


[WDM] - Downloading: 100%|█████████████████████████| 8.08M/8.08M [00:00<00:00, 20.0MB/s]


In [3]:
#assign url and instruct browser to visit it

# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)
#One is that we're searching for elements with a specific combination of tag (div) 
#and attribute (list_text). As an example, ul.item_list would be
#found in HTML as <ul class="item_list">.

True

In [4]:
# setting up HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')
#Notice how we've assigned slide_elem as the variable to look for the <div /> tag and 
#its descendent (the other tags within the <div /> element)? This is our parent element. 
#This means that this element holds all of the other elements within it, 
#and we'll reference it when we want to filter search results even further. 
#The . is used for selecting classes, such as list_text, so the code 'div.list_text' 
#pinpoints the <div /> tag with the class of list_text. CSS works from right to left,
#such as returning the last item on the list instead of the first. Because of this, 
#when using select_one, the first matching element returned will be a <li /> element with 
#a class of slide and all nested elements within it.

In [5]:
#scraping for article title.
slide_elem.find('div', class_='content_title')
                
#this code is looking for the speficic data we're looking for is in a '<div />' with 
# a class of 'content_title'

<div class="content_title">NASA Establishes Board to Initially Review Mars Sample Return Plans</div>

In [6]:
#The title is in that mix of HTML in our output—that's awesome! 
#But we need to get just the text, and the extra HTML stuff isn't necessary. type the following:

# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title


'NASA Establishes Board to Initially Review Mars Sample Return Plans'

In [7]:
#There are two methods used to find tags and attributes with BeautifulSoup:

#.find() is used when we want only the first class and attribute we've specified.
#.find_all() is used when we want to retrieve all of the tags and attributes.
#For example, if we were to use .find_all() instead of .find() when pulling the summary, we would retrieve all of the summaries on the page instead of just the first one.



In [8]:
# Use the parent element to find the summary text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'The board will assist with analysis of current plans and goals for one of the most difficult missions humanity has ever undertaken.'

In [9]:
#next were going to be retrieving images from another website.


### Featured Images


In [10]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)


In [11]:
# Find and click the full image button.
#added the index at the end to set it to click on the second button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [12]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [13]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel


'image/featured/mars3.jpg'

In [14]:
#We've done a lot with that single line.

#Let's break it down:

#An img tag is nested within this HTML, so we've included it.
#.get('src') pulls the link to the image.
#What we've done here is tell BeautifulSoup to look inside the <img /> tag for an image with a class of fancybox-image. Basically we're saying, "This is where the image we want lives—use the link that's inside these tags."



In [15]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

#added the base URL because the image link wont pop up with that link by itself. It needs to 
#be completed

'https://spaceimages-mars.com/image/featured/mars3.jpg'

In [17]:
#the next website, we are going to scrape an entire table
#we're going to use Pandas and use the '.read_html()' function


In [18]:
#set up the code

df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df


#Now let's break it down:

#df = pd.read_htmldf = pd.read_html('https://galaxyfacts-mars.com')[0] With this line, 
#we're creating a new DataFrame from the HTML table. The Pandas function read_html() 
#specifically searches for and returns a list of tables found in the HTML. 
#By specifying an index of 0, we're telling Pandas to pull only the first table it 
#encounters, or the first item in the list. Then, it turns the table into a DataFrame.
#df.columns=['description', 'Mars', 'Earth'] Here, we assign columns to the new 
#DataFrame for additional clarity.
#df.set_index('description', inplace=True) By using the .set_index() function, 
#we're turning the Description column into the DataFrame's index. inplace=True means 
#that the updated index will remain in place, without having to reassign the DataFrame 
#to a new variable.

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [19]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [20]:
browser.quit()

In [22]:
#We are done web scraping but we need to make this proccess automatic by converting it into a 
# .py file. So we need to download the code into python file 
#so we dont have to copy and paste  each cell